In [1]:
# Experimenting with processing of raw data from get info script

# python 3.8.10 environment
import pandas as pd
import numpy as np
import glob
import os
import re
from datetime import datetime

In [3]:
# Read in all text files containing raw info and add to dataframe
folder_path = "../raw_data"

file_paths = glob.glob(os.path.join(folder_path, "*vol*.txt"))

dfs = [pd.read_csv(path, sep=',', header=None, skiprows=1) for path in file_paths]

df_combined = pd.concat(dfs, ignore_index=True)

display(df_combined)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,Barcode Example,Microsoft Corporation,Surface Laptop 4,024541514157,11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz,4,17014460416,30,NaN,Microsoft Windows 11 Pro,The machine is permanently activated.,A,SSD 256060514304 0000000000000000 ...
1,0,Listing Number Example,Microsoft Corporation,Surface Laptop 4,024541514157,11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz,4,17014460416,30,NaN,Microsoft Windows 11 Pro,The machine is permanently activated.,A,SSD 256060514304 0000000000000000 ...
2,1,Test,Microsoft Corporation,Surface Laptop 5,0F00SET23113FB,12th Gen Intel(R) Core(TM) i7-1265U,10,16995729408,35,Perth Mint,Microsoft Windows 11 Pro,The machine is permanently activated.,a,SSD 255060320256 ACE4_2E00_2A7E_D6AE_2EE...
3,1,ss04592,LENOVO,10STS0PK00,PC1536JS,Intel(R) Core(TM) i5-8400 CPU @ 2.80GHz,6,17043300352,26,Perth Mint,Microsoft Windows 11 Pro,The machine is permanently activated.,b,SSD 500107862016 0025_38D3_31A0_CA17.|SS...
4,1,SS04593,LENOVO,10STS0PK00,PC1536K5,Intel(R) Core(TM) i5-8400 CPU @ 2.80GHz,6,17043300352,26,Perth Mint,Microsoft Windows 11 Pro,The machine is permanently activated.,b,SSD 500107862016 0025_38D3_31A0_CA21.|SS...
5,1,SS04589,TOSHIBA,PORTEGE Z30-C,ZG032986H,Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz,2,8501653504,24,NaN,Microsoft Windows 10 Pro,The machine is permanently activated.,c,SSD 128035676160 86PS1232TG5W |SSD ...
6,1,SS04590,HP,HP EliteBook x360 1030 G3,5CD929345T,Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz,4,8425525248,29,NaN,Microsoft Windows 10 Pro,The machine is permanently activated.,b,SSD 256060514304 0025_3886_9103_ACA5.|SS...
7,1,SS04591,TOSHIBA,PORTEGE Z30-C,4H102307H,Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz,2,8501653504,24,NaN,Microsoft Windows 10 Pro,The machine is permanently activated.,c,SSD 256060514304 37BS10IKT8HT |SSD ...
8,1,SS04583,HP,HP EliteBook x360 1030 G3,5CD9293452,Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz,4,8425525248,29,NaN,Microsoft Windows 11 Pro,The machine is permanently activated.,C,SSD 256060514304 0000000000000000 |SS...
9,1,SS04584,HP,HP EliteBook 830 G6,5CG016351S,Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz,4,8395841536,26,NaN,Microsoft Windows 11 Pro,The machine is permanently activated.,c,SSD 256060514304 E823_8FA6_BF53_0001_001...


In [18]:
# Convert bytes to GB, always rounding UP, and format with 'GB'

df_combined['RAM'] = (np.ceil(df_combined[7] / (1024**3))).astype(int).astype(str) + 'GB'
# display(df_combined)

In [19]:
# Define RAM lookup table
ram_lookup_table = {
    0: "Unknown",
    1: "Other",
    2: "DRAM",
    3: "Synchronous DRAM",
    4: "Cache DRAM",
    5: "EDO",
    6: "EDRAM",
    7: "VRAM",
    8: "SRAM",
    9: "RAM",
    10: "ROM",
    11: "Flash",
    12: "EEPROM",
    13: "FEPROM",
    14: "EPROM",
    15: "CDRAM",
    16: "3DRAM",
    17: "SDRAM",
    18: "SGRAM",
    19: "RDRAM",
    20: "DDR",
    21: "DDR2",
    22: "DDR2 FB-DIMM",
    23: "Reserved",
    24: "DDR3",
    25: "FBD2",
    26: "DDR4",
    27: "LPDDR",
    28: "LPDDR2",
    29: "LPDDR3",
    30: "LPDDR4",
    31: "Logical non-volatile device",
    32: "HBM",
    33: "HBM2",
    34: "DDR5"
}

In [20]:
# Map the numeric codes to names and add to new column for ram amount and type
df_combined['RAM Amount and Type'] = df_combined['RAM'] + ' ' + df_combined[8].map(ram_lookup_table)
# display(df_combined)

In [21]:
# CPU Generation
def ordinal(n: int) -> str:
    """Return an integer as an ordinal string (1 -> 1st, 2 -> 2nd, etc.)."""
    if 10 <= n % 100 <= 20:
        suffix = "th"
    else:
        suffix = {1: "st", 2: "nd", 3: "rd"}.get(n % 10, "th")
    return f"{n}{suffix}"

def get_cpu_generation(cpu_name: str) -> str:
    """
    Determine CPU generation from CPU name string.
    Supports:
      - Intel Core i3/i5/i7/i9 (desktop & mobile)
      - Intel Core Ultra (desktop & mobile, with mobile suffixes)
      - AMD Ryzen (desktop & mobile)
    """
    if not isinstance(cpu_name, str):
        return "Unknown"

    cpu_lower = cpu_name.lower()

    # # ---- Intel Core i3/i5/i7/i9 (desktop & mobile) ----
    # Capture the CPU tier (i3/i5/i7/i9) and the digits
    match = re.search(r'(i[3579])-([0-9]{3,5})', cpu_lower)
    
    if match:
        tier = match.group(1)   # e.g. i7
        digits = match.group(2) # e.g. 1185
        num = int(digits)

        # Determine generation
        if num < 1000:
            gen = 1
        elif num < 10000:
            gen = int(str(num)[0])  # 2nd to 9th gen
        else:
            gen = int(str(num)[:2])  # 10th gen+

        if '11th gen' in cpu_lower:
            return f"{tier} 11th Gen"
        elif '12th gen' in cpu_lower:
            return f"{tier} 12th Gen"
        return f"{tier} {ordinal(gen)} Gen"

    # ---- Intel Core Ultra (desktop & mobile) ----
    # Handles: "Core(TM) Ultra" or "Core Ultra"
    # Allows dash or space, and optional suffixes like U, H, KF, K, etc.
    ultra_match = re.search(
        r'core(\(tm\))?\s+ultra\s+(\d)[-\s](\d+)([a-z]{0,3})?', cpu_lower
    )
    if ultra_match:
        return "15th Gen"

    # Match Ryzen tier (3, 5, 7, 9) and model number
    ryzen_match = re.search(r'ryzen\s+([3579])\s+([0-9]{4,5})', cpu_lower)

    if ryzen_match:
        tier = ryzen_match.group(1)            # e.g., "5"
        model = ryzen_match.group(2)           # e.g., "5800"
        series = model[:1] + "000"       # e.g., "5" → "5000"

        return f"Ryzen {tier} {series} Series"
    return "Unknown"

In [ ]:
df_combined["CPU Type"] = df_combined[5].apply(get_cpu_generation)
# display(df_combined)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,RAM,RAM Amount and Type,CPU Type
0,1,Barcode Example,Microsoft Corporation,Surface Laptop 4,024541514157,11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz,4,17014460416,30,NaN,Microsoft Windows 11 Pro,The machine is permanently activated.,A,SSD 256060514304 0000000000000000 ...,16GB,16GB LPDDR4,i7 11th Gen
1,0,Listing Number Example,Microsoft Corporation,Surface Laptop 4,024541514157,11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz,4,17014460416,30,NaN,Microsoft Windows 11 Pro,The machine is permanently activated.,A,SSD 256060514304 0000000000000000 ...,16GB,16GB LPDDR4,i7 11th Gen
2,1,Test,Microsoft Corporation,Surface Laptop 5,0F00SET23113FB,12th Gen Intel(R) Core(TM) i7-1265U,10,16995729408,35,Perth Mint,Microsoft Windows 11 Pro,The machine is permanently activated.,a,SSD 255060320256 ACE4_2E00_2A7E_D6AE_2EE...,16GB,NaN,i7 12th Gen
3,1,ss04592,LENOVO,10STS0PK00,PC1536JS,Intel(R) Core(TM) i5-8400 CPU @ 2.80GHz,6,17043300352,26,Perth Mint,Microsoft Windows 11 Pro,The machine is permanently activated.,b,SSD 500107862016 0025_38D3_31A0_CA17.|SS...,16GB,16GB DDR4,i5 8th Gen
4,1,SS04593,LENOVO,10STS0PK00,PC1536K5,Intel(R) Core(TM) i5-8400 CPU @ 2.80GHz,6,17043300352,26,Perth Mint,Microsoft Windows 11 Pro,The machine is permanently activated.,b,SSD 500107862016 0025_38D3_31A0_CA21.|SS...,16GB,16GB DDR4,i5 8th Gen
5,1,SS04589,TOSHIBA,PORTEGE Z30-C,ZG032986H,Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz,2,8501653504,24,NaN,Microsoft Windows 10 Pro,The machine is permanently activated.,c,SSD 128035676160 86PS1232TG5W |SSD ...,8GB,8GB DDR3,i5 6th Gen
6,1,SS04590,HP,HP EliteBook x360 1030 G3,5CD929345T,Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz,4,8425525248,29,NaN,Microsoft Windows 10 Pro,The machine is permanently activated.,b,SSD 256060514304 0025_3886_9103_ACA5.|SS...,8GB,8GB LPDDR3,i5 8th Gen
7,1,SS04591,TOSHIBA,PORTEGE Z30-C,4H102307H,Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz,2,8501653504,24,NaN,Microsoft Windows 10 Pro,The machine is permanently activated.,c,SSD 256060514304 37BS10IKT8HT |SSD ...,8GB,8GB DDR3,i5 6th Gen
8,1,test1,ASUSTeK COMPUTER INC.,ROG Strix G513IE_G513IE,N3NRKD03324310B,AMD Ryzen 7 4800H with Radeon Graphics,8,16558182400,26,NaN,Microsoft Windows 11 Home,The machine is permanently activated.,a,SSD 512110190592 0000_0000_0100_0000_E4D...,16GB,16GB DDR4,Ryzen 7 4000 Series
9,0,test2,ASUSTeK COMPUTER INC.,ROG Strix G513IE_G513IE,N3NRKD03324310B,AMD Ryzen 7 4800H with Radeon Graphics,8,16558182400,26,NaN,Microsoft Windows 11 Home,The machine is permanently activated.,b,Unspecified 1500301909504 NA4BD4C1 ...,16GB,16GB DDR4,Ryzen 7 4000 Series


In [ ]:
for col in df_combined.select_dtypes(include=['object']).columns:
    df_combined[col] = df_combined[col].str.strip()
# display(df_combined)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,RAM,RAM Amount and Type,CPU Type
0,1,Barcode Example,Microsoft Corporation,Surface Laptop 4,024541514157,11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz,4,17014460416,30,NaN,Microsoft Windows 11 Pro,The machine is permanently activated.,A,SSD 256060514304 0000000000000000 ...,16GB,16GB LPDDR4,i7 11th Gen
1,0,Listing Number Example,Microsoft Corporation,Surface Laptop 4,024541514157,11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz,4,17014460416,30,NaN,Microsoft Windows 11 Pro,The machine is permanently activated.,A,SSD 256060514304 0000000000000000 ...,16GB,16GB LPDDR4,i7 11th Gen
2,1,Test,Microsoft Corporation,Surface Laptop 5,0F00SET23113FB,12th Gen Intel(R) Core(TM) i7-1265U,10,16995729408,35,Perth Mint,Microsoft Windows 11 Pro,The machine is permanently activated.,a,SSD 255060320256 ACE4_2E00_2A7E_D6AE_2EE...,16GB,NaN,i7 12th Gen
3,1,ss04592,LENOVO,10STS0PK00,PC1536JS,Intel(R) Core(TM) i5-8400 CPU @ 2.80GHz,6,17043300352,26,Perth Mint,Microsoft Windows 11 Pro,The machine is permanently activated.,b,SSD 500107862016 0025_38D3_31A0_CA17.|SS...,16GB,16GB DDR4,i5 8th Gen
4,1,SS04593,LENOVO,10STS0PK00,PC1536K5,Intel(R) Core(TM) i5-8400 CPU @ 2.80GHz,6,17043300352,26,Perth Mint,Microsoft Windows 11 Pro,The machine is permanently activated.,b,SSD 500107862016 0025_38D3_31A0_CA21.|SS...,16GB,16GB DDR4,i5 8th Gen
5,1,SS04589,TOSHIBA,PORTEGE Z30-C,ZG032986H,Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz,2,8501653504,24,NaN,Microsoft Windows 10 Pro,The machine is permanently activated.,c,SSD 128035676160 86PS1232TG5W |SSD ...,8GB,8GB DDR3,i5 6th Gen
6,1,SS04590,HP,HP EliteBook x360 1030 G3,5CD929345T,Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz,4,8425525248,29,NaN,Microsoft Windows 10 Pro,The machine is permanently activated.,b,SSD 256060514304 0025_3886_9103_ACA5.|SS...,8GB,8GB LPDDR3,i5 8th Gen
7,1,SS04591,TOSHIBA,PORTEGE Z30-C,4H102307H,Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz,2,8501653504,24,NaN,Microsoft Windows 10 Pro,The machine is permanently activated.,c,SSD 256060514304 37BS10IKT8HT |SSD ...,8GB,8GB DDR3,i5 6th Gen
8,1,test1,ASUSTeK COMPUTER INC.,ROG Strix G513IE_G513IE,N3NRKD03324310B,AMD Ryzen 7 4800H with Radeon Graphics,8,16558182400,26,NaN,Microsoft Windows 11 Home,The machine is permanently activated.,a,SSD 512110190592 0000_0000_0100_0000_E4D...,16GB,16GB DDR4,Ryzen 7 4000 Series
9,0,test2,ASUSTeK COMPUTER INC.,ROG Strix G513IE_G513IE,N3NRKD03324310B,AMD Ryzen 7 4800H with Radeon Graphics,8,16558182400,26,NaN,Microsoft Windows 11 Home,The machine is permanently activated.,b,Unspecified 1500301909504 NA4BD4C1 ...,16GB,16GB DDR4,Ryzen 7 4000 Series


In [24]:
# Only keep first 4 words for Ryzen
keyword = 'AMD Ryzen'
mask = df_combined[5].str.contains(keyword, case=False, na=False)

df_combined['CPU'] = df_combined.apply(
    lambda row: ' '.join(row[5].split()[:4]) if mask[row.name] else row[5],
    axis=1
)
# display(df_combined)

In [ ]:
# Replace Intel(R) Core(TM) with Intel Core and keep the next word (model)
df_combined['CPU'] = df_combined['CPU'].str.replace(
    r'.*Intel\(R\) Core\(TM\)\s+(\S+).*',
    r'Intel Core \1',
    regex=True
)
# display(df_combined)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,RAM,RAM Amount and Type,CPU Type,CPU
0,1,Barcode Example,Microsoft Corporation,Surface Laptop 4,024541514157,11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz,4,17014460416,30,NaN,Microsoft Windows 11 Pro,The machine is permanently activated.,A,SSD 256060514304 0000000000000000 ...,16GB,16GB LPDDR4,i7 11th Gen,Intel Core i7-1185G7
1,0,Listing Number Example,Microsoft Corporation,Surface Laptop 4,024541514157,11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz,4,17014460416,30,NaN,Microsoft Windows 11 Pro,The machine is permanently activated.,A,SSD 256060514304 0000000000000000 ...,16GB,16GB LPDDR4,i7 11th Gen,Intel Core i7-1185G7
2,1,Test,Microsoft Corporation,Surface Laptop 5,0F00SET23113FB,12th Gen Intel(R) Core(TM) i7-1265U,10,16995729408,35,Perth Mint,Microsoft Windows 11 Pro,The machine is permanently activated.,a,SSD 255060320256 ACE4_2E00_2A7E_D6AE_2EE...,16GB,NaN,i7 12th Gen,Intel Core i7-1265U
3,1,ss04592,LENOVO,10STS0PK00,PC1536JS,Intel(R) Core(TM) i5-8400 CPU @ 2.80GHz,6,17043300352,26,Perth Mint,Microsoft Windows 11 Pro,The machine is permanently activated.,b,SSD 500107862016 0025_38D3_31A0_CA17.|SS...,16GB,16GB DDR4,i5 8th Gen,Intel Core i5-8400
4,1,SS04593,LENOVO,10STS0PK00,PC1536K5,Intel(R) Core(TM) i5-8400 CPU @ 2.80GHz,6,17043300352,26,Perth Mint,Microsoft Windows 11 Pro,The machine is permanently activated.,b,SSD 500107862016 0025_38D3_31A0_CA21.|SS...,16GB,16GB DDR4,i5 8th Gen,Intel Core i5-8400
5,1,SS04589,TOSHIBA,PORTEGE Z30-C,ZG032986H,Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz,2,8501653504,24,NaN,Microsoft Windows 10 Pro,The machine is permanently activated.,c,SSD 128035676160 86PS1232TG5W |SSD ...,8GB,8GB DDR3,i5 6th Gen,Intel Core i5-6300U
6,1,SS04590,HP,HP EliteBook x360 1030 G3,5CD929345T,Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz,4,8425525248,29,NaN,Microsoft Windows 10 Pro,The machine is permanently activated.,b,SSD 256060514304 0025_3886_9103_ACA5.|SS...,8GB,8GB LPDDR3,i5 8th Gen,Intel Core i5-8250U
7,1,SS04591,TOSHIBA,PORTEGE Z30-C,4H102307H,Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz,2,8501653504,24,NaN,Microsoft Windows 10 Pro,The machine is permanently activated.,c,SSD 256060514304 37BS10IKT8HT |SSD ...,8GB,8GB DDR3,i5 6th Gen,Intel Core i5-6300U
8,1,test1,ASUSTeK COMPUTER INC.,ROG Strix G513IE_G513IE,N3NRKD03324310B,AMD Ryzen 7 4800H with Radeon Graphics,8,16558182400,26,NaN,Microsoft Windows 11 Home,The machine is permanently activated.,a,SSD 512110190592 0000_0000_0100_0000_E4D...,16GB,16GB DDR4,Ryzen 7 4000 Series,AMD Ryzen 7 4800H
9,0,test2,ASUSTeK COMPUTER INC.,ROG Strix G513IE_G513IE,N3NRKD03324310B,AMD Ryzen 7 4800H with Radeon Graphics,8,16558182400,26,NaN,Microsoft Windows 11 Home,The machine is permanently activated.,b,Unspecified 1500301909504 NA4BD4C1 ...,16GB,16GB DDR4,Ryzen 7 4000 Series,AMD Ryzen 7 4800H


In [26]:

df_combined['CPU'] = df_combined['CPU'] + ' ' + df_combined[6].astype(str) + ' Core Processor'
# display(df_combined)

In [27]:
df_combined['Windows Type and Version'] = df_combined[10].str.replace('Microsoft', '', regex=False)
# display(df_combined)

In [28]:
# Windows Activated column
df_combined['Is Windows Activated?'] = df_combined[11].replace({'The machine is permanently activated.': 'Yes', 'n': 'No'})
# display(df_combined)

In [29]:
# Rows where col 0 == 0
df_system = df_combined[df_combined[0] == 1].reset_index(drop=True)

# Rows where col 0 == 1
df_listing = df_combined[df_combined[0] == 0].reset_index(drop=True)

In [ ]:
# separate disks frame
disks = df_listing[13].str.split('|', expand=True)
# display(disks)

,0,1,2
0,SSD 256060514304 0000000000000000 ...,SSD 256060514304 0100_0000_0000_0000_8CE...,None
1,Unspecified 1500301909504 NA4BD4C1 ...,SSD 512110190592 0000_0000_0100_0000_...,None
2,Unspecified 1500301909504 0000000000000000 ...,SSD 512110190592 0000_0000_0100_0000_...,Unspecified 1500301909504 0000000000000000


In [ ]:
df_system.rename(columns={1: 'Barcode', 2: 'Manufacturer', 3: 'Model', 4: 'Serial', 9: 'Corporate Supplier', 12: 'Grade'}, inplace=True)
df_system = df_system.drop([0, 5, 6, 7, 8, 10, 11, 13, 'RAM', 'RAM Amount and Type', 'CPU', 'Windows Type and Version', 'Is Windows Activated?'], axis=1)
# display(df_system)

,Barcode,Manufacturer,Model,Serial,Corporate Supplier,Grade,CPU Type
0,Barcode Example,Microsoft Corporation,Surface Laptop 4,024541514157,NaN,A,i7 11th Gen
1,Test,Microsoft Corporation,Surface Laptop 5,0F00SET23113FB,Perth Mint,a,i7 12th Gen
2,ss04592,LENOVO,10STS0PK00,PC1536JS,Perth Mint,b,i5 8th Gen
3,SS04593,LENOVO,10STS0PK00,PC1536K5,Perth Mint,b,i5 8th Gen
4,SS04589,TOSHIBA,PORTEGE Z30-C,ZG032986H,NaN,c,i5 6th Gen
5,SS04590,HP,HP EliteBook x360 1030 G3,5CD929345T,NaN,b,i5 8th Gen
6,SS04591,TOSHIBA,PORTEGE Z30-C,4H102307H,NaN,c,i5 6th Gen
7,test1,ASUSTeK COMPUTER INC.,ROG Strix G513IE_G513IE,N3NRKD03324310B,NaN,a,Ryzen 7 4000 Series
8,SS04583,HP,HP EliteBook x360 1030 G3,5CD9293452,NaN,C,i5 8th Gen
9,SS04584,HP,HP EliteBook 830 G6,5CG016351S,NaN,c,i5 8th Gen


In [ ]:
# Foramat Grade column
def map_grade(grade):
    grade = str(grade).strip().upper()
    if grade == 'A':
        return 'Grade A'
    elif grade == 'B':
        return 'Grade B'
    else:
        return 'Grade C'

df_system['Grade'] = df_system['Grade'].apply(map_grade)
# display(df_system)

,Barcode,Manufacturer,Model,Serial,Corporate Supplier,Grade,CPU Type
0,Barcode Example,Microsoft Corporation,Surface Laptop 4,024541514157,NaN,Grade A,i7 11th Gen
1,Test,Microsoft Corporation,Surface Laptop 5,0F00SET23113FB,Perth Mint,Grade A,i7 12th Gen
2,ss04592,LENOVO,10STS0PK00,PC1536JS,Perth Mint,Grade B,i5 8th Gen
3,SS04593,LENOVO,10STS0PK00,PC1536K5,Perth Mint,Grade B,i5 8th Gen
4,SS04589,TOSHIBA,PORTEGE Z30-C,ZG032986H,NaN,Grade C,i5 6th Gen
5,SS04590,HP,HP EliteBook x360 1030 G3,5CD929345T,NaN,Grade B,i5 8th Gen
6,SS04591,TOSHIBA,PORTEGE Z30-C,4H102307H,NaN,Grade C,i5 6th Gen
7,test1,ASUSTeK COMPUTER INC.,ROG Strix G513IE_G513IE,N3NRKD03324310B,NaN,Grade A,Ryzen 7 4000 Series
8,SS04583,HP,HP EliteBook x360 1030 G3,5CD9293452,NaN,Grade C,i5 8th Gen
9,SS04584,HP,HP EliteBook 830 G6,5CG016351S,NaN,Grade C,i5 8th Gen


In [ ]:
# Clean up barcode letters
df_system['Barcode'] = df_system['Barcode'].str.upper()
# display(df_system)

,Barcode,Manufacturer,Model,Serial,Corporate Supplier,Grade,CPU Type
0,BARCODE EXAMPLE,Microsoft Corporation,Surface Laptop 4,024541514157,NaN,Grade A,i7 11th Gen
1,TEST,Microsoft Corporation,Surface Laptop 5,0F00SET23113FB,Perth Mint,Grade A,i7 12th Gen
2,SS04592,LENOVO,10STS0PK00,PC1536JS,Perth Mint,Grade B,i5 8th Gen
3,SS04593,LENOVO,10STS0PK00,PC1536K5,Perth Mint,Grade B,i5 8th Gen
4,SS04589,TOSHIBA,PORTEGE Z30-C,ZG032986H,NaN,Grade C,i5 6th Gen
5,SS04590,HP,HP EliteBook x360 1030 G3,5CD929345T,NaN,Grade B,i5 8th Gen
6,SS04591,TOSHIBA,PORTEGE Z30-C,4H102307H,NaN,Grade C,i5 6th Gen
7,TEST1,ASUSTeK COMPUTER INC.,ROG Strix G513IE_G513IE,N3NRKD03324310B,NaN,Grade A,Ryzen 7 4000 Series
8,SS04583,HP,HP EliteBook x360 1030 G3,5CD9293452,NaN,Grade C,i5 8th Gen
9,SS04584,HP,HP EliteBook 830 G6,5CG016351S,NaN,Grade C,i5 8th Gen


In [34]:
manufacturer_map = {
    'microsoft': 'Microsoft',
    'acer' : 'Acer',
    'apple' : 'Apple',
    'asus' : 'Asus',
    'toshiba': 'Toshiba',
    'dell': 'Dell',
    'hp': 'HP',
    'lenovo': 'Lenovo',
    'msi' : 'MSI',
    'samsung': 'Samsung',
    'sony': 'Sony'
}

In [35]:
# Clean up Manufacturer column
# Step 1: Apply known mappings
for keyword, name in manufacturer_map.items():
    df_system.loc[df_system['Manufacturer'].str.contains(keyword, case=False, na=False), 'Manufacturer'] = name

# Step 2: Set unmatched values to "Other"
df_system['Manufacturer'] = df_system['Manufacturer'].where(
    df_system['Manufacturer'].isin(manufacturer_map.values()), 'Other'
)

In [ ]:
# Clean up NaN and add ID column
df_system = df_system.fillna('')
df_system = df_system.astype(str)
df_system.insert(loc=0, column='ID', value=None) # can change value
# display(df_system)

,ID,Barcode,Manufacturer,Model,Serial,Corporate Supplier,Grade,CPU Type
0,None,BARCODE EXAMPLE,Microsoft,Surface Laptop 4,024541514157,,Grade A,i7 11th Gen
1,None,TEST,Microsoft,Surface Laptop 5,0F00SET23113FB,Perth Mint,Grade A,i7 12th Gen
2,None,SS04592,Lenovo,10STS0PK00,PC1536JS,Perth Mint,Grade B,i5 8th Gen
3,None,SS04593,Lenovo,10STS0PK00,PC1536K5,Perth Mint,Grade B,i5 8th Gen
4,None,SS04589,Toshiba,PORTEGE Z30-C,ZG032986H,,Grade C,i5 6th Gen
5,None,SS04590,HP,HP EliteBook x360 1030 G3,5CD929345T,,Grade B,i5 8th Gen
6,None,SS04591,Toshiba,PORTEGE Z30-C,4H102307H,,Grade C,i5 6th Gen
7,None,TEST1,Asus,ROG Strix G513IE_G513IE,N3NRKD03324310B,,Grade A,Ryzen 7 4000 Series
8,None,SS04583,HP,HP EliteBook x360 1030 G3,5CD9293452,,Grade C,i5 8th Gen
9,None,SS04584,HP,HP EliteBook 830 G6,5CG016351S,,Grade C,i5 8th Gen


In [ ]:
# Rearrange Columns
df_system = df_system.reindex(columns=['ID', 'Manufacturer', 'Model', 'Serial', 'CPU Type', 'Barcode', 'Grade', 'Corporate Supplier'])
# display(df_system)

,ID,Manufacturer,Model,Serial,CPU Type,Barcode,Grade,Corporate Supplier
0,None,Microsoft,Surface Laptop 4,024541514157,i7 11th Gen,BARCODE EXAMPLE,Grade A,
1,None,Microsoft,Surface Laptop 5,0F00SET23113FB,i7 12th Gen,TEST,Grade A,Perth Mint
2,None,Lenovo,10STS0PK00,PC1536JS,i5 8th Gen,SS04592,Grade B,Perth Mint
3,None,Lenovo,10STS0PK00,PC1536K5,i5 8th Gen,SS04593,Grade B,Perth Mint
4,None,Toshiba,PORTEGE Z30-C,ZG032986H,i5 6th Gen,SS04589,Grade C,
5,None,HP,HP EliteBook x360 1030 G3,5CD929345T,i5 8th Gen,SS04590,Grade B,
6,None,Toshiba,PORTEGE Z30-C,4H102307H,i5 6th Gen,SS04591,Grade C,
7,None,Asus,ROG Strix G513IE_G513IE,N3NRKD03324310B,Ryzen 7 4000 Series,TEST1,Grade A,
8,None,HP,HP EliteBook x360 1030 G3,5CD9293452,i5 8th Gen,SS04583,Grade C,
9,None,HP,HP EliteBook 830 G6,5CG016351S,i5 8th Gen,SS04584,Grade C,


In [ ]:
# Clean up listing df
df_listing.rename(columns={1: 'Listing Number', 3: 'Model'}, inplace=True)
df_listing = df_listing.drop([0, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 'RAM', 'CPU Type'], axis=1)
# display(df_listing)

,Listing Number,Model,RAM Amount and Type,CPU,Windows Type and Version,Is Windows Activated?
0,Listing Number Example,Surface Laptop 4,16GB LPDDR4,Intel Core i7-1185G7 4 Core Processor,Windows 11 Pro,Yes
1,test2,ROG Strix G513IE_G513IE,16GB DDR4,AMD Ryzen 7 4800H 8 Core Processor,Windows 11 Home,Yes
2,test3,ROG Strix G513IE_G513IE,16GB DDR4,AMD Ryzen 7 4800H 8 Core Processor,Windows 11 Home,Yes


In [ ]:
# create dataframe of storage disk info and remove USB drive (serial number is zeros) info
disks = disks.replace(r'.*0{16}.*', np.nan, regex=True)
# display(disks)

,0,1,2
0,NaN,SSD 256060514304 0100_0000_0000_0000_8CE...,NaN
1,Unspecified 1500301909504 NA4BD4C1 ...,SSD 512110190592 0000_0000_0100_0000_...,NaN
2,NaN,SSD 512110190592 0000_0000_0100_0000_...,NaN


In [ ]:
def push_nans_right(row):
    # Sort row values: non-NaNs first, NaNs last
    return row.dropna().tolist() + [np.nan] * (len(row) - row.count())

# clean up NaNs
disks = disks.apply(push_nans_right, axis=1, result_type='expand')

disks = disks.dropna(axis=1, how='all')

# display(disks)

,0,1
0,SSD 256060514304 0100_0000_0000_0000_8CE...,NaN
1,Unspecified 1500301909504 NA4BD4C1 ...,SSD 512110190592 0000_0000_0100_0000_...
2,SSD 512110190592 0000_0000_0100_0000_...,NaN


In [ ]:
# define nice sizes in GB
nice_sizes = [128, 256, 512, 1024, 2048, 4096, 8192]

def snap_to_nice(size_gb):
    # pick the closest nice size
    return min(nice_sizes, key=lambda x: abs(x - size_gb))

def format_storage(cell):
    if pd.isna(cell):
        return np.nan
    parts = str(cell).split()
    if len(parts) < 2:
        return cell
    
    storage_type = parts[0]
    size_bytes = int(parts[1])
    
    # Convert bytes to GB
    size_gb = size_bytes / (1024**3)
    
    # Snap to nearest nice value
    snapped = snap_to_nice(size_gb)
    
    return f"{snapped}GB {storage_type}"

# Apply to every cell
disks_formatted = disks.applymap(format_storage)
# Rename columns to Disk1, Disk2, ...
disks_formatted.columns = [f"Disk{i+1}" for i in range(disks_formatted.shape[1])]
# display(disks_formatted)

,Disk1,Disk2
0,256GB SSD,NaN
1,1024GB Unspecified,512GB SSD
2,512GB SSD,NaN


In [ ]:
df_listing_2 = pd.concat([df_listing, disks_formatted], axis=1)
df_listing_2 = df_listing_2.astype(str)
# display(df_listing_2)

,Listing Number,Model,RAM Amount and Type,CPU,Windows Type and Version,Is Windows Activated?,Disk1,Disk2
0,Listing Number Example,Surface Laptop 4,16GB LPDDR4,Intel Core i7-1185G7 4 Core Processor,Windows 11 Pro,Yes,256GB SSD,nan
1,test2,ROG Strix G513IE_G513IE,16GB DDR4,AMD Ryzen 7 4800H 8 Core Processor,Windows 11 Home,Yes,1024GB Unspecified,512GB SSD
2,test3,ROG Strix G513IE_G513IE,16GB DDR4,AMD Ryzen 7 4800H 8 Core Processor,Windows 11 Home,Yes,512GB SSD,nan


In [ ]:
# Save processed data to csv
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S") 

filename_s = f"system_data_{timestamp}.csv"
filename_l = f"listing_data_{timestamp}.csv"

df_system.to_csv(filename_s, index=False) 
df_listing.to_csv(filename_l, index=False) 